In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

We will load the data first, and then follow exactly the same steps to transform the test set as we did for the train set.
A better way to do this would have been to define functions and apply them to the test set.

In [2]:
test = pd.read_csv('data/test_set.csv', parse_dates=[3])
tube = pd.read_csv("data/tube.csv")
bom = pd.read_csv("bom_final.csv", index_col=0)

In [3]:
test['year'] = test.quote_date.dt.year
test['month'] = test.quote_date.dt.month
test['dayofyear'] = test.quote_date.dt.dayofyear
test['dayofweek'] = test.quote_date.dt.dayofweek
test['day'] = test.quote_date.dt.day

In [4]:
# copy id for submission later on
ids = test.id 

# Drop useless features from test for analysis
test = test.drop(['id', 'quote_date'], axis = 1)

In [5]:
test.head()

,tube_assembly_id,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,year,month,dayofyear,dayofweek,day
0,TA-00001,S-0066,0,0,Yes,1,2013,6,174,6,23
1,TA-00001,S-0066,0,0,Yes,2,2013,6,174,6,23
2,TA-00001,S-0066,0,0,Yes,5,2013,6,174,6,23
3,TA-00001,S-0066,0,0,Yes,10,2013,6,174,6,23
4,TA-00001,S-0066,0,0,Yes,25,2013,6,174,6,23


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30235 entries, 0 to 30234
Data columns (total 11 columns):
tube_assembly_id      30235 non-null object
supplier              30235 non-null object
annual_usage          30235 non-null int64
min_order_quantity    30235 non-null int64
bracket_pricing       30235 non-null object
quantity              30235 non-null int64
year                  30235 non-null int64
month                 30235 non-null int64
dayofyear             30235 non-null int64
dayofweek             30235 non-null int64
day                   30235 non-null int64
dtypes: int64(8), object(3)
memory usage: 2.5+ MB


In [7]:
test = pd.merge(test, tube, on ='tube_assembly_id')

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

test['bracket_pricing'] = le.fit_transform(test.bracket_pricing)
test['end_a_1x'] = le.fit_transform(test.end_a_1x)
test['end_a_2x'] = le.fit_transform(test.end_a_2x)
test['end_x_1x'] = le.fit_transform(test.end_x_1x)
test['end_x_2x'] = le.fit_transform(test.end_x_2x)

In [9]:
end_form = pd.read_csv('data/tube_end_form.csv')

In [10]:
# Lable encode 'forming'
end_form['forming'] = le.fit_transform(end_form.forming)

# Map 'forming' value onto 'end_a' and 'end_x' columns
test['end_a'] = test['end_a'].map(end_form.set_index('end_form_id')['forming']) 
test['end_x'] = test['end_x'].map(end_form.set_index('end_form_id')['forming'])

# Fill null value with 0, as no ends means no forming
test.end_a.fillna(0, inplace=True)
test.end_x.fillna(0, inplace=True)

In [12]:
test = pd.merge(test, bom, on ='tube_assembly_id')
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30235 entries, 0 to 30234
Data columns (total 44 columns):
tube_assembly_id      30235 non-null object
supplier              30235 non-null object
annual_usage          30235 non-null int64
min_order_quantity    30235 non-null int64
bracket_pricing       30235 non-null int32
quantity              30235 non-null int64
year                  30235 non-null int64
month                 30235 non-null int64
dayofyear             30235 non-null int64
dayofweek             30235 non-null int64
day                   30235 non-null int64
material_id           30009 non-null object
diameter              30235 non-null float64
wall                  30235 non-null float64
length                30235 non-null float64
num_bends             30235 non-null int64
bend_radius           30235 non-null float64
end_a_1x              30235 non-null int32
end_a_2x              30235 non-null int32
end_x_1x              30235 non-null int32
end_x_2x            

In [13]:
# Split tube_assembly_id into two seperate columns
tube_id = test['tube_assembly_id'].str.split("-", n=1, expand=True)
test['tube_assembly_id'] = tube_id[1].astype(int)

# Split supplier id into two seperate columns and assign the numbers only
supplier_id = test['supplier'].str.split("-", n=1, expand=True) 
test['supplier'] = supplier_id[1].astype(int)

test.head()

,tube_assembly_id,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,year,month,dayofyear,dayofweek,...,component_id_5,quantity_5,component_id_6,quantity_6,component_id_7,quantity_7,component_id_8,quantity_8,Total_quantity,Total_weight
0,1,66,0,0,1,1,2013,6,174,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.096
1,1,66,0,0,1,2,2013,6,174,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.096
2,1,66,0,0,1,5,2013,6,174,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.096
3,1,66,0,0,1,10,2013,6,174,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.096
4,1,66,0,0,1,25,2013,6,174,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.096


In [14]:
# Align min_order_quantity with 'quantity'
test['min_order_quantity'] = test['min_order_quantity'].where(
    test.min_order_quantity > test.quantity, test.quantity)

In [15]:
test.drop(columns=['material_id'], inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30235 entries, 0 to 30234
Data columns (total 43 columns):
tube_assembly_id      30235 non-null int32
supplier              30235 non-null int32
annual_usage          30235 non-null int64
min_order_quantity    30235 non-null int64
bracket_pricing       30235 non-null int32
quantity              30235 non-null int64
year                  30235 non-null int64
month                 30235 non-null int64
dayofyear             30235 non-null int64
dayofweek             30235 non-null int64
day                   30235 non-null int64
diameter              30235 non-null float64
wall                  30235 non-null float64
length                30235 non-null float64
num_bends             30235 non-null int64
bend_radius           30235 non-null float64
end_a_1x              30235 non-null int32
end_a_2x              30235 non-null int32
end_x_1x              30235 non-null int32
end_x_2x              30235 non-null int32
end_a                 3

In [16]:
# Save test to file

test.to_csv('test_final.csv')